## 1. Le modèle 

### 1.1 Principe

La régression logistique est un méthode d'apprentissage supervisée très utile pour expliquer / prédire une variable discrète (à l'opposé de continu) lorsque le jeu de données comporte de nombreuses variables catégorielles.

### 1.2 Notations

Soit Y la variable à prédire / expliquée (dans notre cas, il s'agit de la souscription ou non à l'option de deposit à long terme) et X = (X1, X2, ..., Xn) les variables explicatives

Dans le cadre de la regression logistique binaire, la variable Y prend deux modalités {0,1}. Les variables $X_{i}$ sont exclusivement binaires ou continues.

- Soit $\Omega$ un ensemble de n échantillons (ici n = 45211) comportant $n_{1}$ (respectivement $n_{0}$)) observations correspondant à la modalité 1 (respectivement 0) de Y (dans notre cas, $n_{1}$ = 5289 et $n_{0}$ = 39922).
- $P(Y=1)$ est la probabilité a priori pour que Y = 1. Pour simplifier on la notera p(1)
- $p(X \vert 1)$ est la distribution conditionnelle des X sachant la valeur prise par Y. 
- La probabilité a posteriori d'obtenir la modalité 1 de Y (resp. 0) sachant la valeur prise par X est $p(1 \vert X)$ (resp. $p(0 \vert X)$)


### 1.3 Le modèle LOGIT

On désigne par le terme LOGIT de $p(1 \vert X)$ l'expression suivante: $\ln {\frac  {p(1\vert X)}{1-p(1\vert X)}}=b_{0}+b_{1}x_{1}+...+b_{n}x_{n}$

- il s'agit bien d'une regression car on cherche à montrer une relation de dépendance entre une variable à expliquer tet une série de variables explicatives
- il s'agit d'une regression logistique car la loi de probabilité est modélisée par une loi logistique (https://fr.wikipedia.org/wiki/Fonction_logistique_(Verhulst))

#### Q.1 Pourquoi la fonction logistique est-elle intéressante ? Notamment par rapport à une fonction affine 

Réponse: Puisque nous avons ici une cominaison linéaire de variables en entrée 
            Fonction affine : utile pour une regression linéaire y = a(x) + b
                => avec plusieurs valeurs de variables en entrée , notre fonctin n'est plus affine.

Expliquer / prédire une variable binaire consiste alors à estimer au mieux les coefficients $b_{i}$ ci-dessus. Ensuite on peut retrouver la probabilité conditionnelle par la formule suivante: 

$p(1\vert X)={\frac  {e^{{b_{0}+b_{1}x_{1}+...+b_{n}x_{n}}}}{1+e^{{b_{0}+b_{1}x_{1}+...+b_{n}x_{n}}}}}$

Il existe plusieurs méthodes de resolutions / d'estimations de ces paramètres (voir principe du maximum de vraisemblance et algo méthode de résolution numérique Newton-Raphson). Nous allons nous concentrer plus sur l'interprétation des résultats dans le cas d'une classification plutot que sur les méthodes de résolution.

## 2. Implémentation

### 2.1 Import des librairies et du jeu de données

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from Preprocessing import preprocessing
import statsmodels.api as sm
from sklearn import linear_model

from sklearn import preprocessing

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

On réutilise le jeu de données sur le marketing dans la banque

### 2.2 Preprocessing des données (voir le package en détails)

Comme précisé plus haut, on a besoin de variables exclusivement binaires ou continues. 

#### Q.2 Quel type d'encoding des variables catégorielles doit-on mettre en place pour ce faire ? 

Réponse: encodage binaire

#### Consigne 1: Mettre en place cet encoding 

Vous pouvez faire cet encoding directement via des instructions dans le notebook. Mais le mieux étant de créer un package dans un script python en dehors du notebook pour faire ce processing et appeler ensuite ce package dans le notebook (comme si vous appeliez un package de pandas ou numpy)

In [14]:
new_df.sample(frac=1)

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y_no,y_yes
26396,57,412,20,123,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
16167,32,268,22,102,5,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
13581,31,418,9,219,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
4101,44,2,19,429,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
4426,35,1861,20,297,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,34,1157,21,103,2,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
26005,30,631,19,174,2,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
37400,40,528,13,340,2,357,5,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
6236,32,-383,27,241,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0


#### Consigne 2: Séparer l'échantillon de données en base d'apprentissage de base de test (70% / 30%)

In [15]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()
%matplotlib inline

#### Répartition naïve

In [9]:
On considere la dataset en :
    - 2/3 apprentissage
    - 1/3 test
    
    ==> qu'on appelera proportion t

SyntaxError: invalid syntax (<ipython-input-9-6fb160be0610>, line 1)

In [16]:
import random
def generate_dataset(n, t):
    return [1 if random.random() < t else 0 for i in range(0,n)]

ens = generate_dataset(4000, 0.01)
sum(ens)

37

Et on divise en base d’apprentissage et de test.

In [17]:
def custom_split_train_test(ens, p):
    choice = generate_dataset(len(ens), p)
    train = [x for x, c in zip(ens, choice) if c == 1]
    test = [x for x, c in zip(ens, choice) if c == 0]
    return train, test

train, test = custom_split_train_test(ens, 0.66)
print(len(train), sum(train), sum(train)/len(train))
print(len(test), sum(test), sum(test)/len(test))

2652 24 0.00904977375565611
1348 13 0.009643916913946587


#### On recommence un grand nombre de fois et on représente la proportion obtenue dans la base de test.

In [18]:
tirages = [sum(test)/len(test) for train, test in [custom_split_train_test(ens, 0.66) for i in range(0,100)]]

#### Repartition Stratifiée

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
new_df.columns

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown', 'y_no', 'y_yes'],
      dtype='object')

In [31]:
# create training and testing vars

#y = new_df.y_yes
#X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.3)

X_train,X_test,y_train,y_test=train_test_split(new_df,y_yes,test_size=0.30,random_state=0)

NameError: name 'y_yes' is not defined

In [ ]:
X = new_df.drop(['quality', 'color'], axis=1)

#### Consigne 3: Utiliser la fonction de scikit learn permettant de mettre en place une régression logistique pour entrainer un modèle 

#### we will fit the model on the training data and try to predict the test data


In [30]:
# fit a model

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()


#logreg.fit(X_train,y_train)

#### Consigne 4: Faire des prédictions sur l'échantillon de test

In [28]:
predictions = logreg.predict(X_test)

NameError: name 'X_test' is not defined

In [ ]:
predictions[0:5]

In [ ]:
predictions

#### Consigne 5: Créez une matrice de confusion pour analyser les résultats

In [ ]:
from sklearn import metrics

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, predictions)

In [ ]:
cnf_matrix

In [ ]:
import seaborn as sns


class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

#### Q.3: Combien a-t'on de TP (True Positif), TN (True Negatif), FP (False Positive) et FN (False Negative) ? 

#### Q.4 Quelle est l'accuracy du modèle ? Sa précision ? Son recall ? Son F-1 score ?

#### Consigne 6: Construire une courbe ROC et une courbe Precision - Recall à partir des résultats

#### Consigne 7: Faire des prédictions sur l'échantillon d'apprentissage et calculer toutes les métriques ci-dessus

#### Consigne 8: Comparer avec les résultats des prédictions sur l'échantillon d'apprentissage et l'échantillon de test 

#### Q.5 Observe t'on un phénomène particulier ? 

#### Bonus: modifier les hyperparamètres du modèle pour essayer d'obtenir de meilleurs résultats

In [ ]:
new_df.head()

In [ ]:
new_df.apply(rija)

In [ ]:
new_df.apply(rija)

In [ ]:
df_new = pd.read_csv("new_df.csv")

In [ ]:
pd.get_dummies(df_new)